In [1]:
import sys
sys.path

['/Users/qzhou/repos/AC2021/solutions',
 '/Library/Frameworks/Python.framework/Versions/3.10/lib/python310.zip',
 '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10',
 '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/lib-dynload',
 '',
 '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages',
 '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/IPython/extensions',
 '/Users/qzhou/.ipython']

In [2]:
import platform
print(platform.sys.version)

3.10.0 (v3.10.0:b494f5935c, Oct  4 2021, 14:59:19) [Clang 12.0.5 (clang-1205.0.22.11)]


### Day18-Part1/2

In [749]:
f = open('../inputs/input18.txt').read().splitlines()

In [732]:
import sys
import math
from functools import reduce

In [758]:
f[0]

'[[[[2,2],7],[[9,2],[5,2]]],[4,[[8,9],9]]]'

In [757]:
def parse_number_regular(c, it):
    n = 0
    while c in '0123456789':
        n *= 10
        n += int(c, 10)
        c = next(it)
    return n, c

def parse_number_rec(it):
    c = next(it)
    print('c1', c)
    if c == '[':
        left = parse_number_rec(it)
        print('left', left)
        c = next(it)
    else:
        left, c = parse_number_regular(c, it)
        print('left c', left, c)
    if c != ',':
        raise ValueError
    c = next(it)
    print('c2', c)
    if c == '[':
        right = parse_number_rec(it)
        print('right', right)
        c = next(it)
    else:
        right, c = parse_number_regular(c, it)
        print('right c', right, c)
    if c != ']':
        raise ValueError
    print('left, right', left, right)
    return (left, right)

it = iter(f[0])
parse_number_rec(it)

c1 [
c1 [
c1 [
c1 [
c1 2
left c 2 ,
c2 2
right c 2 ]
left, right 2 2
left (2, 2)
c2 7
right c 7 ]
left, right (2, 2) 7
left ((2, 2), 7)
c2 [
c1 [
c1 9
left c 9 ,
c2 2
right c 2 ]
left, right 9 2
left (9, 2)
c2 [
c1 5
left c 5 ,
c2 2
right c 2 ]
left, right 5 2
right (5, 2)
left, right (9, 2) (5, 2)
right ((9, 2), (5, 2))
left, right ((2, 2), 7) ((9, 2), (5, 2))
left (((2, 2), 7), ((9, 2), (5, 2)))
c2 [
c1 4
left c 4 ,
c2 [
c1 [
c1 8
left c 8 ,
c2 9
right c 9 ]
left, right 8 9
left (8, 9)
c2 9
right c 9 ]
left, right (8, 9) 9
right ((8, 9), 9)
left, right 4 ((8, 9), 9)
right (4, ((8, 9), 9))
left, right (((2, 2), 7), ((9, 2), (5, 2))) (4, ((8, 9), 9))
left ((((2, 2), 7), ((9, 2), (5, 2))), (4, ((8, 9), 9)))


StopIteration: 

In [735]:
def parse_numbers(lines):
    for line in lines:
        it = iter(line)
        if next(it) != '[':
            raise ValueError
        yield parse_number_rec(it)
        
def parse_number_rec(it):
    c = next(it)
    if c == '[':
        left = parse_number_rec(it)
        c = next(it)
    else:
        left, c = parse_number_regular(c, it)
    if c != ',':
        raise ValueError
    c = next(it)
    if c == '[':
        right = parse_number_rec(it)
        c = next(it)
    else:
        right, c = parse_number_regular(c, it)
    if c != ']':
        raise ValueError
    return (left, right)

def parse_number_regular(c, it):
    n = 0
    while c in '0123456789':
        n *= 10
        n += int(c, 10)
        c = next(it)
    return n, c

with open('../inputs/input18.txt', 'r') as f:
    numbers = parse_numbers(map(str.strip, f))
numbers

<generator object parse_numbers at 0x2b4d2e5e0>

In [731]:
import sys
import math
from functools import reduce

def parse_numbers(lines):
    for line in lines:
        it = iter(line)
        if next(it) != '[':
            raise ValueError
        yield parse_number_rec(it)

def parse_number_rec(it):
    c = next(it)
    if c == '[':
        left = parse_number_rec(it)
        c = next(it)
    else:
        left, c = parse_number_regular(c, it)
    if c != ',':
        raise ValueError
    c = next(it)
    if c == '[':
        right = parse_number_rec(it)
        c = next(it)
    else:
        right, c = parse_number_regular(c, it)
    if c != ']':
        raise ValueError
    return (left, right)

def parse_number_regular(c, it):
    n = 0
    while c in '0123456789':
        n *= 10
        n += int(c, 10)
        c = next(it)
    return n, c

def add_numbers(a, b):
    return reduce_number((a, b))

def reduce_number(n):
    reduced = True
    while reduced:
        n, reduced, *_ = explode_rec(n, 0)
        if not reduced:
            n, reduced = split_rec(n)
    return n

def explode_rec(n, level):
    if not isinstance(n, int):
        l, r = n
        if level >= 4:
            return 0, True, l, r
        else:
            l, reduced, expl, expr = explode_rec(l, level + 1)
            if reduced:
                if expr != 0:
                    r = add_left(r, expr)
                    expr = 0
            else:
                r, reduced, expl, expr = explode_rec(r, level + 1)
                if reduced:
                    if expl != 0:
                        l = add_right(l, expl)
                        expl = 0
            if reduced:
                return (l, r), True, expl, expr
    return n, False, 0, 0

def add_left(n, m):
    if isinstance(n, int):
        return n + m
    else:
        a, b = n
        return add_left(a, m), b

def add_right(n, m):
    if isinstance(n, int):
        return n + m
    else:
        a, b = n
        return a, add_right(b, m)

def split_rec(n):
    if isinstance(n, int):
        if n >= 10:
            a = n // 2
            return (a, n - a), True
    else:
        l, r = n
        l, reduced = split_rec(l)
        if not reduced:
            r, reduced = split_rec(r)
        if reduced:
            return (l, r), True
    return n, False

def number_magnitude(n):
    if isinstance(n, int):
        return n
    l, r = n
    return 3 * number_magnitude(l) + 2 * number_magnitude(r)

def part1(in_file):
    with open(in_file, 'r') as f:
        numbers = parse_numbers(map(str.strip, f))
        numbers = map(reduce_number, numbers)
        res = reduce(add_numbers, numbers)
    print(res)
    m = number_magnitude(res)
    print(m)

def part2(in_file):
    with open(in_file, 'r') as f:
        numbers = parse_numbers(map(str.strip, f))
        numbers = list(map(reduce_number, numbers))
    m_max = -math.inf
    for i, n1 in enumerate(numbers):
        for j, n2 in enumerate(numbers):
            if i == j: continue
            m = number_magnitude(add_numbers(n1, n2))
            if m > m_max:
                m_max = m
    print(m_max)

if __name__ == '__main__':
    part1(*sys.argv[1:])
    part2(*sys.argv[1:])



4
4


['[[[[2,2],7],[[9,2],[5,2]]]', '']

In [682]:
re.findall('\[\d,\d\]', f[0])

['[2,2]', '[9,2]', '[5,2]', '[8,9]']

In [683]:
a = [[[[2,2],7],[[9,2],[5,2]]],[4,[[8,9],9]]]

In [685]:
a[1]

[4, [[8, 9], 9]]

### Day17-Part1/2

In [658]:
f = open('../inputs/input17.txt').read().splitlines()
x_range = [int(x) for x in f[0].split(',')[0].split('=')[1].split('..')]
y_range = [int(y) for y in f[0].split(',')[1].split('=')[1].split('..')]
print(x_range, y_range)

['target area: x=155..215, y=-132..-72']


([155, 215], [-132, -72])

In [655]:
def inRange(pos, rangex, rangey):
    return (pos[0] in range(rangex[0], rangex[1]+1)) and (pos[1] in range(rangey[0], rangey[1]+1))

def step(x0, y0, rangex, rangey):
    pos = [(x0, y0)]
    vx, vy = x0, y0
    while True:
        if inRange(pos[-1], rangex, rangey):
            # return max([p[1] for p in pos])
            return (x0, y0)
            # return pos
        if pos[-1][1]<rangey[0]:
            return -1
            # return pos
        if vx > 0:
            vx -= 1
        elif vx < 0:
            vx += 1
        vy -= 1
        pos.append((pos[-1][0]+vx, pos[-1][1]+vy))

In [626]:
# part1:
# at vy>0 velocity, it will reach y-pos=0 with velocity vy-1
# if vy<0, it will miss the target is vy < y-target min
# so, y-target min = -vy - 1 --> vy = -y-taget min -1
# max height= vy * (vy+1)/2

vy = -y_range[0]-1
print('part 1: ', vy * (vy+1)/2)

part 1:  8646.0


In [ ]:
init = []
for xi in range(1, x_range[1]+1):
    for yi in range(y_range[0], -y_range[0]):
        if step(xi, yi, x_range, y_range) != -1:
            init.append(step(xi, yi, x_range, y_range))
print('part 2: ', len(init))

5945

### Day16-Part1/2

In [572]:
with open('../inputs/input16.txt') as f:
    f = f.read().splitlines()[0]
f_bin = ''.join([bin(int(s, 16))[2:].zfill(4) for s in f])

In [576]:
def literal_packet(s):
    p_version, p_type = int(s[0:3],2), int(s[3:6], 2)
    flag = s[6]
    group = s[7:11]
    start = 6
    while flag=='1':
        start += 5
        flag = s[start]
        group += s[start+1:start+5]
    packet = int(group, 2)
    s = s[start+5:]
    return (p_version, p_type, packet, s)

def op0_packet(s):
    p_version, p_type = int(s[0:3],2), int(s[3:6], 2)
    subpacket = s[22:]
    return p_version, p_type, subpacket

def op1_packet(s):
    p_version, p_type = int(s[0:3],2), int(s[3:6], 2)
    subpacket = s[18:]
    return p_version, p_type, subpacket

def unpack(s1):
    list_packet = []

    while len(s1)>0:
        if int(s1[3:6],2) == 4:
            _ver, _type, _value, s1 = literal_packet(s1)
            list_packet.append((_ver, _type, _value))    
        else:
            if s1[6]=='0':
                _ver, _type, s1 = op0_packet(s1)
                list_packet.append((_ver, _type))
            else:
                _ver, _type, s1 = op1_packet(s1)
                list_packet.append((_ver, _type))
        if len(s1)==0 or int(s1, 2)==0:
            break
    return list_packet

unpacked = unpack(f_bin)
print('Part 1:', sum([l[0] for l in unpacked]))

Part 1: 1002


In [577]:
dict_operation = {'0':'+', '1':'*', '2':'min', '3':'max', '5':'gt', '6':'lt', '7':'eq'}
expression = ','.join([str(l[2]) if l[1]==4 else dict_operation[str(l[1])] for l in unpacked])
expression

'+,*,eq,+,8,5,3,+,14,6,10,817,*,2519282017,gt,1133,1133,*,3581,gt,+,9,5,2,+,9,2,13,min,1047014,863,61916,2,min,2474,8,+,61,10,*,2053,gt,+,4,7,3,+,2,15,3,4,max,3071,25896876792,90333271,119514860,53417,*,lt,70133,70133,61068,*,198,eq,2335,5594,+,1866,164,46351,+,*,14,8,10,*,12,7,10,*,3,3,11,*,75308549,lt,1512,32762322674,max,7,*,196,31,33,36,*,lt,3115,45499,216,*,100,104,188,7,199,*,210,+,3162941497,max,819624592872,180,254,708340,max,+,max,+,min,+,max,min,max,+,max,+,min,min,max,*,max,*,+,*,187,11,*,28441,lt,+,13,15,6,+,15,9,5,+,176,4043,149405,1,172,*,90,187,231,*,eq,193831084,1421,218,*,695560,eq,33555,33555,*,+,4,9,3,+,4,7,11,+,12,10,15,5,14,2942,min,13040895,max,2706,236764,*,lt,29093,29093,2639,2905,*,60706885,lt,+,7,8,9,+,14,2,8,*,849193,gt,3660,6045,+,22,821619064094,675,13,66876028,*,48406,gt,7781281650951,3322,max,16157976,9,165,*,gt,166,166,97,15,*,gt,1001,67695326,20632,*,lt,10219686,205,483239986,11674,min,52,535330074729,7666,167,1561,*,gt,50028,144,117,min,120864373,8604,

In [ ]:
from functools import reduce

def calcs(exp0):
    if exp0[0] == '+':
        return sum([int(i) for i in exp0[1:]])
    if exp0[0] == '*':
        return reduce(lambda x, y: x*y, [int(i) for i in exp0[1:]])
    if exp0[0] == 'min':
        return min([int(i) for i in exp0[1:]])
    if exp0[0] == 'max':
        return max([int(i) for i in exp0[1:]])
    if exp0[0] == 'lt':
        return int(int(exp0[1])<int(exp0[2]))
    if exp0[0] == 'gt':
        return int(int(exp0[1])>int(exp0[2]))
    if exp0[0] == 'eq':
        return int(int(exp0[1])==int(exp0[2]))

In [578]:
import math

op = [sum, math.prod, min, max,
      lambda ls: ls[0], # literal
      lambda ls: 1 if ls[0] > ls[1] else 0,  # gt
      lambda ls: 1 if ls[0] < ls[1] else 0,  # lt
      lambda ls: 1 if ls[0] == ls[1] else 0] # eq

bs = f_bin

def ps2(startbit):
    i = startbit # index into bs
    ID = int(bs[i+3:i+6],2) # packet type ID
    i += 6
    if ID == 4: #literal value
        vals = [0]
        while True:
            vals[0] = 16*vals[0] + int(bs[i+1:i+5],2)
            i += 5
            if bs[i-5] == '0': #last value packet
                break
    else:
        vals = []
        if bs[i] == '0': # subpacket length in bits
            endi = i + 16 + int(bs[i+1:i+16],2)
            i += 16
            while i < endi:
                i,v = ps2(i)
                vals.append(v)
        else:
            np = int(bs[i+1:i+12],2) # number of subpackets
            i += 12
            for _ in range(np):
                i,v = ps2(i)
                vals.append(v)

    return i,op[ID](vals)

print('part 2: ', ps2(0)[1])

part 2:  1673210814091


### Day15-Part1/2

In [40]:
import numpy as np

with open('../inputs/input15.txt') as f:
    files = f.read().splitlines()
files = np.array([[int(s) for s in file] for file in files])

In [44]:
def scale(files, j):
    file2 = files
    for i in range(1, 5, 1):
        fi = (files + i)%9
        fi = np.where(fi==0, 9, fi)
        file2 = np.hstack((file2, fi)) # horizontal

    file3 = file2
    for i in range(1, 5, 1):
        fi = (file2 + i)%9
        fi = np.where(fi==0, 9, fi)
        file3 = np.vstack((file3, fi)) # vertical
    return file3
files_large = scale(files, 5)

In [43]:
d=[[int(c) for c in l.strip()] for l in open("../inputs/input15.txt","rt")]

def step(ij,v,p,u,w,N,I):
    # mark_from_ij
    v[ij]=True
    u.remove(ij)
    i,j=ij//N,ij%N
    for ni,nj in ((i-1,j),(i+1,j),(i,j-1),(i,j+1)):
        if 0<=ni<N and 0<=nj<N:
            nij=N*ni+nj
            if v[nij]: continue
            if p[nij]>p[ij]+w[nij]:
                p[nij]=p[ij]+w[nij]
                u.append(nij)
    # min_unvisited
    mij=mp=I
    for uij in u:
        if p[uij]<mp:
            mij=uij; mp=p[uij]
    return mij

def solve(d):
    N=len(d); NN=N*N; I=999999 # infinity :)
    w=[] # weights, 1-dimension copy of input
    for i in range(N): w+=d[i]
    v=[False for j in range(NN)] # visited
    p=[I for j in range(NN)] # paths
    u=[] # unvisited items (ij indexes)
    p[0]=0 # start with top left corner
    u.append(0)
    uij = step(0,v,p,u,w,N,I)
    while uij!=I:
        uij = step(uij,v,p,u,w,N,I)
    return p[NN-1] # right bottom
    # return p

def scale(a,K):
    N=len(a)
    g=[[0 for i in range(K*N)] for j in range(K*N)]
    for ii in range(K):
        for jj in range(K):
            for i in range(N):
                for j in range(N):
                    g[ii*N+i][jj*N+j] = (a[i][j]-1+ii+jj)%9+1
    return g

print( solve(d) )
print( solve(scale(d,5)) )

527
2887


In [341]:
# Solution not working for loop case

filename = 'input15'

import numpy as np

with open('../inputs/'+filename+'.txt') as f:
    files = f.read().splitlines()
files = np.array([[int(s) for s in file] for file in files])
def mysolve(files):

    list_dist = []
    list_index = []
    for i in range(len(files), -len(files), -1):
        list_dist.append(files[:,::-1].diagonal(i).tolist())
        list_index.append(list(zip(np.indices(files.shape)[0][:, ::-1].diagonal(i),\
        np.indices(files.shape)[1][:, ::-1].diagonal(i))))
    list_dist.pop(0)
    list_index.pop(0)

    dict_dist = {}
    for i,j in zip(list_index, list_dist):
        for ii, jj in zip(i,j):
            dict_dist[ii] = jj
    dict_dist[(0,0)] = 0

    def update_node(node, dict_dist):
        i, j = node
        up = (max(i-1, 0), j)
        left = (i, max(j-1, 0))
        if up == node:
            dict_dist[node]+=dict_dist[left]
            return dict_dist
        if left == node:
            dict_dist[node]+=dict_dist[up]
            return dict_dist
        dict_dist[node] += min(dict_dist[up], dict_dist[left])
        return dict_dist

    for i in range(len(files)*2-1):
        for node in list_index[i]:
            dict_dist = update_node(node, dict_dist)
            # print(node, dict_dist[node])
    # return (dict_dist[(files.shape[0]-1, files.shape[1]-1)])
    return dict_dist
               

### Day14-Part1/2

In [129]:
with open('../inputs/input14.txt') as f:
    files = f.read().split('\n\n')

s0 = files[0]

import re
dict_instruct = {}
instruct = re.findall('(\w+) -> (\w+)\n', files[1])
for x,y in instruct:
    dict_instruct[x] = y

In [130]:
from collections import defaultdict

def polymer(s0, dict_instruct, step=1):

    count_instruct = defaultdict(int)

    for i in range(len(s0)-1):
        count_instruct[s0[i:i+2]]+=1
        
    for i in range(step):
        new_count_instruct = count_instruct.copy()
        for j in count_instruct.keys():
            freq = count_instruct[j]
            if freq > 0:
                new_count_instruct[j[0]+dict_instruct[j]] += freq
                new_count_instruct[dict_instruct[j]+j[1]] += freq
                new_count_instruct[j] -= freq
  
        count_instruct = new_count_instruct
    
    dict_cnt = defaultdict(int)
    for k,v in count_instruct.items():
        if v>0:
            dict_cnt[k[0]]+=v
            dict_cnt[k[1]]+=v
    dict_cnt[s0[0]]+=1  # 1st and last char counted only once, all the rest counted twice
    dict_cnt[s0[-1]]+=1
    return max(dict_cnt.values())/2 - min(dict_cnt.values())/2

print('part 1: ', polymer(s0, dict_instruct, step=10))
print('part 2: ', polymer(s0, dict_instruct, step=40))

part 1:  2375.0
part 2:  1976896901756.0


In [131]:
# original looping solution that will freeze at large steps
def polymer(s0, dict_instruct, step=1):
    new_s = s0
    for i in range(step): 
        out_s = ''
        for s in range(len(new_s)-1):
            out_s += new_s[s]+dict_instruct[new_s[s:s+2]]
        out_s += new_s[-1]
        new_s = out_s
    out_count = []
    for i in set(out_s):
        # out_count.append((i, out_s.count(i)))
        out_count.append(out_s.count(i))
    out_count = sorted(out_count)
    return out_count[-1]-out_count[0]           
print('part 1: ', polymer(s0, dict_instruct, step=10))

part 1:  2375


### Day13-Part1/2

In [119]:
with open('../inputs/input13.txt') as f:
    files = f.read().split('\n\n')
coord = [c.split(',') for c in files[0].split('\n')]
coord = [[int(c) for c in cor] for cor in coord]

import re
instruction = re.findall('(\w+)=(\w+)', files[1])

x, y = zip(*coord)

import numpy as np
paper = np.zeros((max(y)+1, max(x)+1))

for c in coord:
    paper[c[1], c[0]] = 1

for step,i in enumerate(instruction):
    fold_line = int(i[1])
    if i[0]=='y':        
        paper[fold_line, :] = 0
        paper_up = paper[0:fold_line, :]
        paper_down = paper[fold_line+1:, :]
        paper_down = paper_down[::-1, :]
        paper = paper_up+paper_down
        print(step+1, i , (paper>0).sum())

    else:
        paper[:, fold_line] = 0
        paper_left = paper[:, 0:fold_line]
        paper_right = paper[:, fold_line+1:]
        paper_right = paper_right[:,::-1]
        paper = paper_left+paper_right
        print(step+1, i, (paper>0).sum())

# Part2 help visualizing
print((paper>0).astype(int))

1 ('x', '655') 704
2 ('y', '447') 595
3 ('x', '327') 495
4 ('y', '223') 422
5 ('x', '163') 351
6 ('y', '111') 294
7 ('x', '81') 250
8 ('y', '55') 215
9 ('x', '40') 178
10 ('y', '27') 149
11 ('y', '13') 123
12 ('y', '6') 103
[[1 0 0 1 0 0 1 1 0 0 0 1 1 0 0 0 0 1 1 0 1 1 1 0 0 1 1 1 1 0 1 0 0 1 0 0
  1 1 0 0]
 [1 0 0 1 0 1 0 0 1 0 1 0 0 1 0 0 0 0 1 0 1 0 0 1 0 1 0 0 0 0 1 0 0 1 0 1
  0 0 1 0]
 [1 1 1 1 0 1 0 0 0 0 1 0 0 1 0 0 0 0 1 0 1 1 1 0 0 1 1 1 0 0 1 1 1 1 0 1
  0 0 0 0]
 [1 0 0 1 0 1 0 1 1 0 1 1 1 1 0 0 0 0 1 0 1 0 0 1 0 1 0 0 0 0 1 0 0 1 0 1
  0 0 0 0]
 [1 0 0 1 0 1 0 0 1 0 1 0 0 1 0 1 0 0 1 0 1 0 0 1 0 1 0 0 0 0 1 0 0 1 0 1
  0 0 1 0]
 [1 0 0 1 0 0 1 1 1 0 1 0 0 1 0 0 1 1 0 0 1 1 1 0 0 1 1 1 1 0 1 0 0 1 0 0
  1 1 0 0]]


### Day12-Part1/2

In [104]:
with open('../inputs/input12.txt') as f:
    files = f.read().splitlines()

# ------------------------------
# create connect point dictionary
# ------------------------------
import re

pairs = []
for t in files:
    pairs += list(re.findall('(\w+)-(\w+)', t))

unique_point = list(set(list(zip(*pairs))[1]) | set(list(zip(*pairs))[0]))

dict_connect = {}
for p in unique_point:
    p_pair = [list(pair) for pair in pairs if p in pair]
    next_point = []
    for s in p_pair:
        next_point += s
    dict_connect[p] = list(set(next_point) - {p} - {'start'})
del dict_connect['end']
print(dict_connect)

# ------------------------------
# create connect point dictionary - alternative online solution
# ------------------------------
# using defaultdict can aovid key error, no need to initiaize a value first
from collections import defaultdict

dict_connect = defaultdict(list)
for f in files:
    pair = f.strip().split('-')
    for p1, p2 in zip(pair, reversed(pair)):
        if p2 != 'start':
            dict_connect[p1].append(p2)
del dict_connect['end']
print(dict_connect)

{'LA': ['end', 'kl', 'tk', 'ed'], 'JR': ['eq'], 'tk': ['LA', 'SQ', 'II', 'cc', 'XW', 'ed'], 'II': ['end', 'tk', 'eq', 'cc', 'ed'], 'SQ': ['tk', 'kl'], 'XW': ['end', 'tk', 'ed', 'eq'], 'start': ['LA', 'kl', 'SQ'], 'kl': ['LA', 'cc', 'SQ'], 'ns': ['eq'], 'eq': ['JR', 'ns', 'II', 'XW', 'ed'], 'cc': ['tk', 'kl', 'II', 'ed'], 'ed': ['LA', 'tk', 'II', 'eq', 'cc', 'XW']}
defaultdict(<class 'list'>, {'XW': ['ed', 'eq', 'tk', 'end'], 'ed': ['XW', 'eq', 'cc', 'LA', 'tk', 'II'], 'cc': ['tk', 'ed', 'kl', 'II'], 'tk': ['cc', 'II', 'XW', 'LA', 'ed', 'SQ'], 'eq': ['ed', 'ns', 'XW', 'II', 'JR'], 'ns': ['eq'], 'LA': ['kl', 'end', 'ed', 'tk'], 'kl': ['LA', 'SQ', 'cc'], 'II': ['tk', 'end', 'cc', 'eq', 'ed'], 'SQ': ['kl', 'tk'], 'start': ['SQ', 'LA', 'kl'], 'JR': ['eq']})


In [105]:
def count(dict_connect, path=['start']):
    count_path = 0
    for node in dict_connect[path[-1]]:
        if node.isupper() or not node in path:
            if node == 'end':
                count_path += 1
                # print('part1: end: ', path, node)
            else:
                count_path += count(dict_connect, path+[node])
    return count_path
        
print('part 1: ', count(dict_connect))

part 1:  3679


In [106]:
def count2(dict_connect, path=['start']):
    count_path = 0
    for node in dict_connect[path[-1]]:
        if node == 'end':
            count_path += 1
            # print('part2: end: ', path, node)
        else:
            if node.islower() and node in path:
                count_path += count(dict_connect, path+[node])
            else:
                count_path += count2(dict_connect, path+[node])
    return count_path
     
print('part 2: ', count2(dict_connect))

part 2:  107395


### Day11-Part1/2

In [169]:
with open('../inputs/input11.txt') as f:
    files = f.read().splitlines()

import numpy as np    
files = np.array([[int(f) for f in file] for file in files])

In [173]:
def radiate(minitoy, i, j):
    for ii in range(max(0, i-1), min(minitoy.shape[0], i+2)):
        for jj in range(max(0, j-1), min(minitoy.shape[1], j+2)):
            minitoy[ii][jj] += 1
    minitoy[i][j] -= 1
    return minitoy

def simulation(minitoy, steps=1):
    max_row, max_col = minitoy.shape
    flash_cnt = 0
    flash_seq = []
    for step in range(steps):
        minitoy = minitoy + 1
        rad_cells = [] # cell radiated already
        new_rad = []        
        while True:
            # print(minitoy)
            for i in range(max_row):
                for j in range(max_col):
                    if minitoy[i][j] > 9:
                        new_rad.append((i, j))
            new_rad = list(set(new_rad) - set(rad_cells))
            # print("new_rad: ", new_rad)
            
            if len(new_rad) == 0:
                break

            for r in new_rad:
                minitoy = radiate(minitoy, r[0], r[1])
            rad_cells = list(set(rad_cells + new_rad))
            new_rad = []
            # print("rad_cells: ", rad_cells)
        
        for cell in rad_cells:
            minitoy[cell[0]][cell[1]] = 0
        flash_cnt += len(rad_cells)
        flash_seq.append(len(rad_cells))

    return minitoy, flash_cnt, flash_seq

print('Part 1: ', simulation(files, steps=100)[1])

s = 100 # random initial number
a, b, c = simulation(files, steps=s)
while max(c) < 100:
    s += 100
    a, b, c = simulation(files, steps=s)
print('Part 2: ', c.index(100)+1)

Part 1:  1591
Part 2:  314


### Day10-Part1/2

In [53]:
with open('../inputs/input10.txt') as f:
    files = f.read().splitlines()

def day10(toy):
    pairs =['<>', '[]', '()', '{}']
    left = ['<', '[', '(', '{']
    illegal = []
    new_toy = toy.copy()    

    for t in toy:
        line = t
        while ('<>' in t) or ('[]' in t) or ('()' in t) or ('{}' in t):
            for p in pairs:
                t = t.replace(p, "")
        for s in left:
            while (s in t):
                t = t.replace(s, "")
        if len(t)>0:
            illegal.append(t[0])
            new_toy.remove(line)

    print('Part 1: ', illegal.count(')')*3 + illegal.count(']')*57 + illegal.count('}')*1197 + illegal.count('>')*25137)
    
    auto_complete = []  
    dict_pairs = {'<':'>', '{':'}', '[':']', '(':')'}
    dict_points = {')':1, ']':2, '}':3, '>':4}

    for t in new_toy:
        while ('<>' in t) or ('[]' in t) or ('()' in t) or ('{}' in t):
            for p in pairs:
                t = t.replace(p, "")
        s = t[::-1]
        auto_complete.append(''.join([dict_pairs[si] for si in s]))

    list_score = []
    for a in auto_complete:
        score = 0
        for ai in a:
            score = 5*score + dict_points[ai]
        list_score.append(score)
    list_score = sorted(list_score)

    print('Part 2: ', list_score[int(len(list_score)/2)])

day10(files)

Part 1:  389589
Part 2:  1190420163


### Day9-Part1/2

In [63]:
with open('../inputs/input9.txt')as f:
    files = f.read().splitlines()
files = [[int(i) for i in t] for t in files]
    
def local_min(toy):    
    local_min = []
    local_min_cor = []
    max_row = len(toy)
    max_col = len(toy[0])

    for i in range(max_row):
        for j in range(max_col):
            right = toy[i][min(j+1, max_col-1)]
            left = toy[i][max(j-1, 0)]
            up = toy[max(0, i-1)][j]
            down = toy[min(max_row-1, i+1)][j]
            if j+1 == max_col:
                right = 10
            if j-1 < 0:
                left = 10
            if i-1 < 0:
                up = 10
            if i+1 == max_row:
                down = 10
            
            if toy[i][j] < min(right, left, up, down):
                local_min.append(toy[i][j])
                local_min_cor.append([i, j])
                
    return sum(local_min)+len(local_min), local_min_cor

def local_basin(toy, cor):    
    new_basin = [(cor[0], cor[1])]
    final_basin = [(cor[0], cor[1])]
    max_row = len(toy)
    max_col = len(toy[0])
    
    while len(new_basin)>0:
        add_basin = []
        for i in range(len(new_basin)):
            x, y = new_basin[i][0], new_basin[i][1]
            # right path
            for j in range(y+1, max_col):
                if ((toy[x][j] == 9) or (toy[x][y] >= toy[x][j])):
                    break
                else:
                    add_basin.append((x, j))
            # left path
            for j in range(y-1, -1, -1):
                if ((toy[x][j] == 9) or (toy[x][y] >= toy[x][j])):
                    break
                else:
                    add_basin.append((x, j))
            # up path
            for i in range(x-1, -1, -1):
                if ((toy[i][y] == 9) or (toy[x][y] >= toy[i][y])):
                    break
                else:
                    add_basin.append((i, y))
            # down path
            for i in range(x+1, max_row):
                if ((toy[i][y] == 9) or (toy[x][y] >= toy[i][y])):
                    break
                else:
                    add_basin.append((i, y))
        add_basin = list(set(add_basin) - set(new_basin))
        final_basin = list(set(final_basin + add_basin))
        new_basin = add_basin
 
    return len(final_basin)
            
sum_min, local_min_cor = local_min(files)
print('Part 1: ', sum_min)

local_basin_area = []
for mcor in local_min_cor:
    local_basin_area.append(local_basin(files, mcor))

local_basin_area = sorted(local_basin_area, reverse=True)[:3]
print('Part 2: ', local_basin_area[0] * local_basin_area[1] * local_basin_area[2])

Part 1:  417
Part 2:  1148965


### Day8-Part1

In [114]:
with open('../inputs/input8.txt')as f:
    files = f.read().splitlines()

import re

signals = []
digits = []

for f in files:
    line = re.findall('[a-z]+', f)
    signals.append(line[:10])
    digits.append(line[-4:])

In [115]:
# part 1
sum4 = 0
display4 = []
for d in digits:
    l = [dd for dd in d if len(dd) in [2, 4, 3, 7]]
    sum4 += len(l)
    display4.append(l)
print(sum4)

239

In [118]:
# part 2
def decode(toy_signals, toy_digits):

    output = []
    dict_digits = {}
    dict_char = {}
    dict_convert = {}
    
    for d,o in zip(toy_signals, toy_digits): 
        for dd in d:   
            if len(dd)==2:
                dict_digits['1'] = ''.join(dd)
            elif len(dd) == 4:
                dict_digits['4'] = ''.join(dd)
            elif len(dd) == 3:
                dict_digits['7'] = ''.join(dd)
            elif len(dd) == 7:
                dict_digits['8'] = ''.join(dd)

        
        new_str = ''.join(d)

        for c in ['a', 'b', 'c', 'd', 'e', 'f', 'g']:
            dict_char[c] = new_str.count(c)

        for i,j in dict_char.items():        
            if j == 9:
                dict_convert['f'] = i
            elif j == 4:
                dict_convert['e'] = i
            elif j == 6:
                dict_convert['b'] = i
                
        dict_convert['a'] = list(set(dict_digits['7']) - set(dict_digits['1']))[0]
        dict_convert['c'] = list(set(dict_digits['1']) - set(dict_convert['f']))[0]
        dict_convert['d'] = list(set(dict_digits['4']) - set(dict_convert['f']) - set(dict_convert['b']) - set(dict_convert['c']))[0]
        dict_convert['g'] = list(set(dict_digits['8']) - set(dict_digits['4']) - set(dict_convert['a']) - set(dict_convert['e']))[0]
        dict_digits['6'] = ''.join(list(set(dict_digits['8']) - set(dict_convert['c'])))
        dict_digits['9'] = ''.join(list(set(dict_digits['8']) - set(dict_convert['e'])))
        dict_digits['0'] = ''.join(list(set(dict_digits['8']) - set(dict_convert['d'])))
        dict_digits['5'] = ''.join(list(set(dict_digits['8']) - set(dict_convert['c']) - set(dict_convert['e'])))
        dict_digits['3'] = ''.join(list(set(dict_digits['8']) - set(dict_convert['b']) - set(dict_convert['e'])))
        dict_digits['2'] = ''.join(list(set(dict_digits['8']) - set(dict_convert['b']) - set(dict_convert['f'])))
        
        decoder = {}
        for i,j in dict_digits.items():
            dict_digits[i] = ''.join(sorted(j))
            decoder[dict_digits[i]] = i

        o = [''.join(sorted(i)) for i in o]
        out = ''.join([decoder[i] for i in o])
        output.append(int(out))

    return output
        
print(sum(decode(signals, digits)))
    

946346


### Day7-Part1/2

In [23]:
with open('../inputs/input7.txt')as f:
    files = f.read().split(",")
files = [int(f) for f in files]

import statistics
import numpy as np

# part 1
median = statistics.median(files)
print('part 1: ', int(sum(abs(np.array(files)-median))))

# part 2
fuel = []
for i in range(min(files), max(files)):
    step = abs(np.array(files)-i)
    fuel.append(sum(step * (1+step)/2))
print('part 2: ', int(min(fuel)))

part 1:  326132
part 2:  88612508


## Day6-Part1/2

In [16]:
with open('../inputs/input6.txt')as f:
    files = f.read().split(",")
files = [int(f) for f in files]

def growfish(fishes, days):   
    for i in range(days):
        newfish = []
        babyfish = []
        for f in fishes:
            newfish.append(f-1)
            if newfish[-1] == -1:
                newfish[-1] = 6
                babyfish.append(8)
        fishes = newfish + babyfish
    return len(fishes)

print(growfish(files, 80))
# print(growfish(files, 256))

359999


In [18]:
def solve(values, days):
    """ O(n) solution 
        instead of storing fish, we can store a count for days till reproduction
        so fish = {"0_days": n, "1_days": n2, "2_days": n3, ...., "10_days":, n11}

    Parameters
    ----------
        values : [list of days till reproduction]
    """

    fish = {}
    for i in range(11):
        fish[str(i)] = 0
    for i in values:
        fish[str(i)] += 1

    for i in range(days):
        new_fish = {}
        new_fish['0'] = fish['1']
        new_fish['1'] = fish['2']
        new_fish['2'] = fish['3']
        new_fish['3'] = fish['4']
        new_fish['4'] = fish['5']
        new_fish['5'] = fish['6']
        new_fish['6'] = fish['7']
        new_fish['7'] = fish['8']
        new_fish['6'] += fish['0']
        new_fish['8'] = fish['0']
        fish = new_fish

    return sum(fish.values())
print(solve(files, 256))

1631647919273


## Day5-Part1/2

In [247]:
with open('../inputs/input5.txt') as f:
    files = f.read().splitlines()

files = [f.split(" ->") for f in files]
files = [[f.strip().split(",") for f in file] for file in files]
files = [[[int(f) for f in ff] for ff in file] for file in files]

In [250]:
point1, point2 = zip(*files)

line_h = []
line_v = []
line_d = []
for p1, p2 in zip(point1, point2):
    if p1[0] == p2[0]:
        line_h.append([p1, p2]) # horziontal line
    if p1[1] == p2[1]:
        line_v.append([p1, p2]) # vertical line
    if abs(p1[0]-p2[0]) == abs(p1[1]-p2[1]):
        line_d.append([p1, p2]) # diagonal line

# generating diagonal line points
point_d = []
for i in range(len(line_d)):
    x1, y1, x2, y2 = line_d[i][0][0], line_d[i][0][1], line_d[i][1][0], line_d[i][1][1]
    xy_points = []
    for i in range(abs(x2-x1)+1):
        x_step = (x2-x1)/abs(x2-x1)
        y_step = (y2-y1)/abs(y2-y1)
        xy_points.append((x1+i*x_step, y1+i*y_step))
    point_d.append(xy_points)
point_d = tuple(point_d)

# generating horizontal line points
point_h = []
for i in range(len(line_h)):
    y1, y2 = line_h[i][0][1], line_h[i][1][1]
    y_points = list(range(min(y1, y2), max(y1, y2)+1))
    xy_points = []
    for j in y_points:
        xy_points.append((line_h[i][0][0], j))
    point_h.append(xy_points)
point_h = tuple(point_h)

# generating vertical line points
point_v = []
for i in range(len(line_v)):
    x1, x2 = line_v[i][0][0], line_v[i][1][0]
    x_points = list(range(min(x1, x2), max(x1, x2)+1))
    xy_points = []
    for j in x_points:
        xy_points.append((j, line_v[i][0][1]))
    point_v.append(xy_points)
point_v = tuple(point_v)

# part 1

point_all = point_h+point_v
point_unique = []
for p in point_all:
    point_unique += p
point_unique = set(point_unique)

# find points only in 1 line
common = []
single_set = set(point_all[0])
for p in point_all[1:]:
    pset = set(p) - set(common)    
    ci = single_set & pset
    if len(ci) > 0:
        common += list(ci)
    single_set ^= pset # keep only exist in either one

print('part1: ', len(point_unique)-len(single_set))

# part 2

point_all = point_h+point_v+point_d
point_unique = []
for p in point_all:
    point_unique += p
point_unique = set(point_unique)

# find points only in 1 line
common = []
single_set = set(point_all[0])
for p in point_all[1:]:
    pset = set(p) - set(common)    
    ci = single_set & pset
    if len(ci) > 0:
        common += list(ci)
    single_set ^= pset # keep only exist in either one

print('part2: ', len(point_unique)-len(single_set))

part1:  6564
part2:  19172


In [252]:
# online solution

with open('../inputs/input5.txt') as f:
    data = f.read()

import re
from collections import Counter

def solution(data, part=1):
    lines = re.findall('(\d+),(\d+) -> (\d+),(\d+)', data)
    points = Counter()
    for line in lines:
        x1, y1, x2, y2 = map(int, line)
        if part == 1 and x1 != x2 and y1 != y2:  # diagonal line
            continue
        dx, dy = x2 - x1, y2 - y1
        length = max(abs(dx), abs(dy))
        x_step, y_step = dx//length, dy//length
        points.update((x1 + i*x_step, y1 + i*y_step) for i in range(length+1))
    return sum(count > 1 for count in points.values())

print('Part 1:', solution(data, part=1))
print('Part 2:', solution(data, part=2))

Part 1: 6564
Part 2: 19172


### Day4-Part1/2

In [478]:
# my solution

with open('../inputs/input4-number.txt') as f:
    number = f.read().splitlines()
number = [n.split(",") for n in number][0]

with open('../inputs/input4-board.txt') as f:
    board = f.read().splitlines()

board1 = [] # prepare each board

n = int((len(board)+1)/6) # number of boards

for i in range(n):
    bi = board[i*6]
    for j in range(i*6+1, i*6+6):
        if j < len(board):
            bi = bi + " " + board[j]
    board1.append(bi)

board1 = [b1.split(" ") for b1 in board1]
# remove empty string
for i, b1 in enumerate(board1):
    board1[i] = [bi for bi in b1 if len(bi)>0]

# run the game

def check_ni_boardi(list_l, number_i, mark):
    if number_i in list_l:
        mark[list_l.index(number_i)] = 1
    return mark

def update_flag(mark):
    flag = [0]*10
    for i in range(5):
        flag[i] = sum(mark[i*5:(i+1)*5]) # horizontal
    for i in range(5, 10):
        flag[i] = sum([mark[ind] for ind in range(i-5, 25, 5)]) # vertical
    return flag 

def run1_board(boardi, number):
    mark = [0]*25
    seq = 0
    for ni in number:
        mark = check_ni_boardi(boardi, ni, mark)
        flag = update_flag(mark)
        if max(flag) == 5:
            break
        seq = seq + 1
    unmarked = [int(j) for i,j in zip(mark, boardi) if i==0]
    score = int(ni) * sum(unmarked) 
    return seq, score

out = []
for b1 in board1:
    out.append(run1_board(b1, number))

# part 1 - get first winning score

rounds, scores= zip(*out)
print(scores[rounds.index(min(rounds))])

# part 2 - get last winning score

rounds, scores= zip(*out)
print(scores[rounds.index(max(rounds))])

41668
10478


In [ ]:
# Online solution I found
# -- much more effient in loading data and do logics

with open('../inputs/input4.txt') as f:
    numbers, *boards = f.read().split('\n\n')
    numbers = list(map(int, numbers.split(',')))
    boards = [[[int(n) for n in row.split()] for row in board.splitlines()] for board in boards]

from itertools import chain

def find_win_score():
    called = []
    for num in numbers:
        called.append(num)
        for board in boards:
            if any(set(line) < set(called) for line in chain(board, zip(*board))):
                unmarked = {n for row in board for n in row} - set(called)
                return sum(unmarked) * num

def find_losing_score():
    called = numbers.copy()
    while called:
        last = called.pop()
        for board in boards:
            if not any(set(line) < set(called) for line in chain(board, zip(*board))):
                unmarked = {n for row in board for n in row} - {last, *called}
                return sum(unmarked) * last
print(find_win_score())
print(find_losing_score())

41668
10478


### Day3-Part1

In [18]:
with open('../inputs/input3.txt') as f:
    lines = f.read().splitlines()

for l in lines:
    ll.append([li for li in l])

In [34]:
from statistics import mode

gamma = []
for i in range(len(ll[0])):
    xi = []
    for lli in ll:
        xi.append(lli[i])        
    gamma.append(mode(xi))
gamma = [int(i) for i in gamma]
epsilon = [1-i for i in gamma]
gamma = int(''.join([str(i) for i in gamma]), 2)
epsilon = int(''.join([str(i) for i in epsilon]), 2)
print(gamma, epsilon, gamma*epsilon)

654 3441 2250414


### Day3-Part2

In [68]:
def findO2(l):
    
    def getListByOne(l, i):
        xi = []
        for lli in l:
            xi.append(lli[i])
        if xi.count('1') >= xi.count('0'):
            pos = [i for i, x in enumerate(xi) if x == "1"]
        else:
            pos = [i for i, x in enumerate(xi) if x == "0"]

        out = []
        for p in pos:
            out.append(l[p])
        return out, len(out)

    subl = l
    lenl = len(l)
    i = 0
    while lenl > 1:
        subl, lenl = getListByOne(subl, i)
        i += 1
    
    if lenl==1:
        O2 = int(''.join([str(i) for i in subl[0]]), 2)
    return O2


def findCO2(l):
    
    def getListByOne(l, i):
        xi = []
        for lli in l:
            xi.append(lli[i])
        if xi.count('0') <= xi.count('1'):
            pos = [i for i, x in enumerate(xi) if x == "0"]
        else:
            pos = [i for i, x in enumerate(xi) if x == "1"]

        out = []
        for p in pos:
            out.append(l[p])
        return out, len(out)

    subl = l
    lenl = len(l)
    i = 0
    while lenl > 1:
        subl, lenl = getListByOne(subl, i)
        i += 1
    
    if lenl==1:
        CO2 = int(''.join([str(i) for i in subl[0]]), 2)
    return CO2

O2 = findO2(ll)
CO2 = findCO2(ll)
print(O2, CO2, O2*CO2)        
    

1935 3145 6085575


### Day2-Part1

In [25]:
with open('../inputs/input2.txt') as f:
    lines = f.read().splitlines()

split_lines = [l.split(" ") for l in lines]
direction, position = map(list, zip(*split_lines))
position = [int(i) for i in position]
depth = 0
pos = 0
for d, p in zip(direction, position):
    if d == 'forward':
        pos += p
    if d == 'up':
        depth -= p
    if d == 'down':
        depth += p
print(depth*pos)

2019945


### Day2-Part2

In [26]:
aim = 0
hori = 0
depth = 0
for d, p in zip(direction, position):
    if d == 'down':
        aim += p
    if d == 'up':
        aim -= p
    if d == 'forward':
        hori += p
        depth += aim * p
print(hori*depth)

1599311480


### Day1-Part1

In [28]:
with open('../inputs/input1.txt') as f:
    lines = f.read().splitlines()

lines = [int(l) for l in lines]

import numpy as np

line_array = np.array(lines)
line_array2 = np.array(lines[1:])
line_array2 = np.append(line_array2, line_array[-1])

sum((line_array2-line_array)>0)

1298

In [45]:
# or just roll XD
sum((np.roll(line_array, -1)[:-1]-line_array[:-1])>0)

1298

### Day1-Part2

In [46]:
line_array2 = np.array(lines[1:])
line_array2 = np.append(line_array2, 0)
line_array3 = np.array(lines[2:])
line_array3 = np.append(line_array3, [0, 0])
line_sum = (line_array + line_array2 + line_array3)[:-2]

sum((np.roll(line_sum, -1)[:-1]-line_sum[:-1])>0)

1248

In [47]:
# double roll
line_sum = line_array[:-2] + np.roll(line_array, -1)[:-2] + np.roll(line_array, -2)[:-2] 
sum((np.roll(line_sum, -1)[:-1]-line_sum[:-1])>0)

1248